<a href="https://colab.research.google.com/github/linneasandersen/dalas-project/blob/main/DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   Get data
- Wikipedia
- OEC
- FAO
2.   Store data in pandas
3. Inspect




In [44]:
import requests
import pandas as pd
from pathlib import Path
import bs4
import lxml
import pandas
import urllib
import kagglehub
import os

from urllib import request

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Countries

In [ ]:
# Countries from kaggle

path = kagglehub.dataset_download("emolodov/country-codes-alpha2-alpha3")

filename = "Country Codes Alpha-2 Alpha-3.csv"

file_path = os.path.join(path, filename)

df_countries = pd.read_csv(file_path)

print(df_countries.head())

Using Colab cache for faster access to the 'country-codes-alpha2-alpha3' dataset.
          country alpha2 alpha3  numeric
0     Afghanistan     AF    AFG        4
1         Albania     AL    ALB        8
2         Algeria     DZ    DZA       12
3  American Samoa     AS    ASM       16
4         Andorra     AD    AND       20


# OEC

In [ ]:
# Define the years to retrieve data for
years = range(1995, 2024)

In [43]:
# Iterate and fetch export data
df_trade = []
url = "https://api-v2.oec.world/tesseract/data.jsonrecords"

for year in years:
    params = {
        "cube": "trade_i_baci_a_92",
        "locale": "en",
        "drilldowns": "Exporter Country Official,Year,HS4",
        "properties": "Exporter Country Official.Exporter Country ISO2",
        "measures": "Trade Value",
        "include": f"Year:{year}"
    }
    r = requests.get(url, params=params)
    try:
        df = pd.DataFrame(r.json()["data"])
        if year == 2000:
            print(r.json())
        df_trade.append(df)
    except KeyError:
        print(f"No data found for export in {year}")


# concatenate
df_trade = pd.concat(df_trade)

print("Finished fetching export data.")

Buffered data was truncated after reaching the output size limit.

In [ ]:
# display data
display(df_trade)
print(df_trade.columns)


# more possibilities for columns, see cubeOECinfo.json

Data set size is order of magnitude 10^6 for both import and export
Currently 6 columns, but more possibilities

## Harmonized system of products

In [ ]:
data_dir = Path("/content/drive/MyDrive/DALAS/data/raw")
data_path_products = data_dir / "hs_product_classification.csv"

In [ ]:
print(data_path_products)
df_HS = pd.read_csv(
    data_path_products,
    sep=';',        # semicolon separator
    quotechar='"'
)
print(df_HS.head())
print(df_HS.columns)

# query where Product Level = HS4
df_HS4 = df_HS[df_HS['Product Level'] == 'HS4']
print(df_HS4.head())
print(df_HS4.shape)

# FAO


In [ ]:
data_dir = Path("/content/drive/MyDrive/DALAS/data/raw")

zip_files = ["Environment_Temperature_change_E_All_Data.zip", "Production_Crops_Livestock_E_All_Data.zip", "Forestry_E_All_Data.zip"]
zip_files = [data_dir / file for file in zip_files]

zip_file = data_dir / "Environment_Temperature_change_E_All_Data.zip"

In [ ]:
from pathlib import Path
from zipfile import ZipFile

for zip_file in zip_files:
  extract_dir = data_dir / "extracted"
  extract_dir.mkdir(parents=True, exist_ok=True)

  # Extract all files
  with ZipFile(zip_file, 'r') as zip_ref:
      zip_ref.extractall(extract_dir)

  print(f"Files extracted to {extract_dir}")


Files extracted to /content/drive/MyDrive/DALAS/data/raw/extracted
Files extracted to /content/drive/MyDrive/DALAS/data/raw/extracted
Files extracted to /content/drive/MyDrive/DALAS/data/raw/extracted


In [ ]:
# Find extracted Excel files
files = list(extract_dir.glob("*.csv"))

for file in files:
    print(file.name)

Production_Crops_Livestock_E_All_Data.csv
Production_Crops_Livestock_E_All_Data_NOFLAG.csv
Production_Crops_Livestock_E_AreaCodes.csv
Production_Crops_Livestock_E_Elements.csv
Production_Crops_Livestock_E_Flags.csv
Production_Crops_Livestock_E_ItemCodes.csv
Environment_Temperature_change_E_All_Data.csv
Environment_Temperature_change_E_Flags.csv
Environment_Temperature_change_E_Elements.csv
Environment_Temperature_change_E_AreaCodes.csv
Environment_Temperature_change_E_All_Data_NOFLAG.csv
Forestry_E_All_Data.csv
Forestry_E_All_Data_NOFLAG.csv
Forestry_E_AreaCodes.csv
Forestry_E_Elements.csv
Forestry_E_Flags.csv
Forestry_E_ItemCodes.csv


In [ ]:
# print columns of all datasets, size of table
for file in files:
    print(file.name)
    if file.name == "Environment_Temperature_change_E_All_Data.csv":
        df_FAO_env = pd.read_csv(file)
    if file.name == "Production_Crops_Livestock_E_All_Data.csv":
        df_FAO_prod = pd.read_csv(file)
    if file.name == "Forestry_E_All_Data.csv":
        df_FAO_forest = pd.read_csv(file)
    df = pd.read_csv(file)
    print(df.shape)
    print(df.columns)
    print("---------------------------------------------------------------------------------")


Production_Crops_Livestock_E_All_Data.csv


/tmp/ipython-input-3814059025.py:7: DtypeWarning: Columns (11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59,62,65,68,71,74,77,80,83,86,89,92,95,98,101,104,107,110,113,116,119,122,125,128,131,134,137,140,143,146,149,152,155,158,161,164,167,170,173,176,179,182,185,188,191,194,197) have mixed types. Specify dtype option on import or set low_memory=False.
  df_FAO_prod = pd.read_csv(file)
/tmp/ipython-input-3814059025.py:10: DtypeWarning: Columns (11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59,62,65,68,71,74,77,80,83,86,89,92,95,98,101,104,107,110,113,116,119,122,125,128,131,134,137,140,143,146,149,152,155,158,161,164,167,170,173,176,179,182,185,188,191,194,197) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


(78164, 198)
Index(['Area Code', 'Area Code (M49)', 'Area', 'Item Code', 'Item Code (CPC)',
       'Item', 'Element Code', 'Element', 'Unit', 'Y1961',
       ...
       'Y2020N', 'Y2021', 'Y2021F', 'Y2021N', 'Y2022', 'Y2022F', 'Y2022N',
       'Y2023', 'Y2023F', 'Y2023N'],
      dtype='object', length=198)
---------------------------------------------------------------------------------
Production_Crops_Livestock_E_All_Data_NOFLAG.csv
(78164, 72)
Index(['Area Code', 'Area Code (M49)', 'Area', 'Item Code', 'Item Code (CPC)',
       'Item', 'Element Code', 'Element', 'Unit', 'Y1961', 'Y1962', 'Y1963',
       'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971',
       'Y1972', 'Y1973', 'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979',
       'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987',
       'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995',
       'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 'Y2002', 'Y2003'

In [ ]:
# prepare to melt
print(df_FAO_env.head())
print(df_FAO_prod.head())
print(df_FAO_forest.head())


   Area Code Area Code (M49)         Area  Months Code    Months  \
0          2            '004  Afghanistan         7001   January   
1          2            '004  Afghanistan         7001   January   
2          2            '004  Afghanistan         7002  February   
3          2            '004  Afghanistan         7002  February   
4          2            '004  Afghanistan         7003     March   

   Element Code             Element Unit  Y1961 Y1961F  ...  Y2020 Y2020F  \
0          7271  Temperature change   °c  0.792      E  ... -0.622      E   
1          6078  Standard Deviation   °c  1.964      E  ...  1.964      E   
2          7271  Temperature change   °c -1.750      E  ...  2.565      E   
3          6078  Standard Deviation   °c  2.606      E  ...  2.606      E   
4          7271  Temperature change   °c  0.483      E  ...  0.647      E   

   Y2021 Y2021F  Y2022 Y2022F  Y2023 Y2023F  Y2024 Y2024F  
0  0.740      E  2.297      E -2.418      E  4.396      E  
1  1.964

In [ ]:
df_FAOs_processed = []
df_FAOs = [df_FAO_env, df_FAO_prod, df_FAO_forest]

for i in range(3):
  df_FAO = df_FAOs[i]

  if i == 1:
    id_vars = ['Area Code', 'Area Code (M49)', 'Area', 'Element Code', 'Element', 'Unit', 'Item Code', 'Item', 'Item Code (CPC)']
  elif i == 2:
    id_vars = ['Area Code', 'Area Code (M49)', 'Area', 'Element Code', 'Element', 'Unit', 'Item Code', 'Item']
  else:
    id_vars = ['Area Code', 'Area Code (M49)', 'Area', 'Months Code', 'Months', 'Element Code', 'Element', 'Unit']

  # 3. Melt the DataFrame
  # This turns all other columns into rows
  df_melted = df_FAO.melt(id_vars=id_vars, var_name='Year_Flag_Raw', value_name='Value_Raw')

  print("\n--- Step 1: After Melting ---")
  print(df_melted.head())

  # 4. Separate Year and Type (Value vs. Flag)
  # Use regex to extract the 4-digit year
  df_melted['Year'] = df_melted['Year_Flag_Raw'].str.extract(r'(\d{4})')

  # Identify if the original column was a value (e.g., 'Y1961') or a flag (e.g., 'Y1961F')
  df_melted['Type'] = df_melted['Year_Flag_Raw'].apply(lambda x: 'Flag' if x.endswith('F') else 'Value')

  print("\n--- Step 2: After Separating Year and Type ---")
  print(df_melted.head())

  # 5. Pivot the table to create separate 'Value' and 'Flag' columns
  # We group by all the original IDs plus the new 'Year'
  df_FAO_processed = df_melted.pivot_table(
      index=id_vars + ['Year'],  # Group by the original IDs and the new 'Year'
      columns='Type',          # Create new columns from 'Type' (i.e., 'Value' and 'Flag')
      values='Value_Raw',      # Use 'Value_Raw' as the cell values
      aggfunc='first'          # Use 'first' since there's only one value per group
  ).reset_index()

  # Clean up the column names
  df_FAO_processed.columns.name = None

  print("\n--- Final Reshaped DataFrame ---")
  print(df_FAO_processed.head())
  print(df_FAO_processed.shape)

  df_FAOs_processed.append(df_FAO_processed)

print(df_FAOs_processed)


--- Step 1: After Melting ---
   Area Code Area Code (M49)         Area  Months Code    Months  \
0          2            '004  Afghanistan         7001   January   
1          2            '004  Afghanistan         7001   January   
2          2            '004  Afghanistan         7002  February   
3          2            '004  Afghanistan         7002  February   
4          2            '004  Afghanistan         7003     March   

   Element Code             Element Unit Year_Flag_Raw Value_Raw  
0          7271  Temperature change   °c         Y1961     0.792  
1          6078  Standard Deviation   °c         Y1961     1.964  
2          7271  Temperature change   °c         Y1961     -1.75  
3          6078  Standard Deviation   °c         Y1961     2.606  
4          7271  Temperature change   °c         Y1961     0.483  

--- Step 2: After Separating Year and Type ---
   Area Code Area Code (M49)         Area  Months Code    Months  \
0          2            '004  Afghanistan 

In [ ]:
print(df_FAOs_processed[0].head)
print(df_FAOs_processed[1].head)
print(df_FAOs_processed[2].head)

df_FAO_proc_env = df_FAOs_processed[0]
df_FAO_proc_prod = df_FAOs_processed[1]
df_FAO_proc_forest = df_FAOs_processed[2]

# export data to csv


<bound method NDFrame.head of         Area Code Area Code (M49)     Area  Months Code               Months  \
0               1            '051  Armenia         7001              January   
1               1            '051  Armenia         7001              January   
2               1            '051  Armenia         7001              January   
3               1            '051  Armenia         7001              January   
4               1            '051  Armenia         7001              January   
...           ...             ...      ...          ...                  ...   
572351       5873            '198     OECD         7020  Meteorological year   
572352       5873            '198     OECD         7020  Meteorological year   
572353       5873            '198     OECD         7020  Meteorological year   
572354       5873            '198     OECD         7020  Meteorological year   
572355       5873            '198     OECD         7020  Meteorological year   

        E

9656 rows, 136 columns

# Wikipedia



In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_countries_by_tariff_rate"
r = request.Request(url, headers={"User-Agent": "Mozilla/5.0"})

request_text = request.urlopen(r).read()
page = bs4.BeautifulSoup(request_text, "lxml")
print(page.find("title"))

In [ ]:
tableau_participants = page.find('table', {'class' : 'wikitable'})
table_body = tableau_participants.find('tbody')
rows = table_body.find_all('tr')

header_row1 = rows[0].find_all(['th', 'td'])
header_row2 = rows[1].find_all(['th', 'td'])

header1_text = [cell.get_text(strip=True) for cell in header_row1]
header2_text = [cell.get_text(strip=True) for cell in header_row2]

combined_headers = [header1_text[0]]
combined_headers.append(f"{header1_text[1]} - {header2_text[0]}")
combined_headers.append(f"{header1_text[1]} - {header2_text[1]}")
combined_headers.append(f"{header1_text[2]} - {header2_text[2]}")
combined_headers.append(f"{header1_text[2]} - {header2_text[3]}")
combined_headers.append(f"{header1_text[3]} - {header2_text[4]}")
combined_headers.append(f"{header1_text[3]} - {header2_text[5]}")

data_rows = rows[2:]

results = []
expected_cols = len(combined_headers)

for row in data_rows:
    cols = row.find_all(['th', 'td'])
    cols = [ele.text.strip() for ele in cols]

    if len(cols) > 3 and cols[3] == "EU member":
        if len(cols) >= 6:
            corrected_cols = cols[:4]
            corrected_cols.append(None)
            corrected_cols.append(cols[4])
            corrected_cols.append(cols[5] if len(cols) > 5 else None)
            cols = corrected_cols
        else:
             print(f"Warning: Row with 'EU member' has unexpected structure: {cols}")
             while len(cols) < expected_cols:
                cols.append(None)


    while len(cols) < expected_cols:
        cols.append(None)
    if len(cols) > expected_cols:
         print(f"Warning: Row still has more than {expected_cols} columns after processing: {cols}")
         cols = cols[:expected_cols]


    results.append(cols)

df = pd.DataFrame(data=results, columns=combined_headers)

display(df.head())
display(df[df['WTO[2][6] - Simple average applied MFN tariff, all products'] == 'EU member'].head())

In [ ]:
# clean headers to remove references, i.e [1]
df.columns = df.columns.str.replace(r'\[\d+\]', '', regex=True)
display(df.head())

In [ ]:
# Separate dataframes into 3
df_WB = df.iloc[:, :3].copy()

# col 1, 4, 5
df_WTO = df.iloc[:, [0, 3, 4]].copy()

# col 1, 6, 7
df_UNCTAD = df.iloc[:, [0, 5, 6]].copy()

# Replace empty strings with NaN to be recognized by dropna
df_WB.replace('', pd.NA, inplace=True)
df_WTO.replace('', pd.NA, inplace=True)
df_UNCTAD.replace('', pd.NA, inplace=True)

# remove rows where 2nd or 3rd col is empty
df_WB = df_WB.dropna(subset=[df_WB.columns[1], df_WB.columns[2]])
df_WTO = df_WTO.dropna(subset=[df_WTO.columns[1], df_WTO.columns[2]])
df_UNCTAD = df_UNCTAD.dropna(subset=[df_UNCTAD.columns[1], df_UNCTAD.columns[2]])

# shapes
print(df_WB.shape)
print(df_WTO.shape)
print(df_UNCTAD.shape)

# Display the new DataFrames
display(df_WB.head())
display(df_WTO.head())
display(df_UNCTAD.head())

In [ ]:
# what years are covered
df_WB = df_WB.rename(columns={'WB - Year': 'Year'})
df_WTO = df_WTO.rename(columns={'WTO - Year': 'Year'})
df_UNCTAD = df_UNCTAD.rename(columns={'UNCTAD - Year': 'Year'})

print(df_WB.columns)
print(df_WTO.columns)
print(df_UNCTAD.columns)
print("Years covered in WB data:")
print(df_WB['Year'].unique())

print("\nYears covered in WTO data:")
print(df_WTO['Year'].unique())

print("\nYears covered in UNCTAD data:")
print(df_UNCTAD['Year'].unique())

# WTO

In [ ]:
# https://apiportal.wto.org/signin
SUBSCRIPTION_KEY = ""

In [ ]:
# indicators
indicator_url = 'http://api.wto.org/timeseries/v1/indicators'

params = {
    'i': 'all',
    'fmt': 'json'
}

headers = {
    'Ocp-Apim-Subscription-Key': SUBSCRIPTION_KEY
}

print(f"Querying WTO API with parameters:\n{params}\n")

try:
    response = requests.get(indicator_url, params=params, headers=headers)

    # --- 4. CHECK THE RESPONSE ---
    if response.status_code == 200:
        print("Success! Data received.\n")
        # Convert the JSON response text into a Python dictionary
        data = response.json()
        print(data)

        # dataframe
        df_i = pd.DataFrame(data)
        print(df_i.head())

    else:
        # Print error details if something went wrong
        print(f"Error: {response.status_code}")
        print(f"Response content: {response.text}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred with the request: {e}")

In [ ]:
# print all available indicators as markdown
print(df_i[['name', 'code']].to_markdown())

In [ ]:
import requests
import json

base_url = "https://api.wto.org/timeseries/v1/data"

params = {
    'i': 'HS_A_0010',  # Indicator
    'r': '840',         # EXAMPLE Reporter: USA
    'pc': 'HS4',
    'ps': '2023-2025',
    'fmt': 'json',      # Format: json
    'lang': '1'         # Language: 1 for English
}

headers = {
    'Ocp-Apim-Subscription-Key': SUBSCRIPTION_KEY
}

print(f"Querying WTO API with parameters:\n{params}\n")

try:
    response = requests.get(base_url, params=params, headers=headers)

    # --- 4. CHECK THE RESPONSE ---
    if response.status_code == 200:
        print("Success! Data received.\n")
        # Convert the JSON response text into a Python dictionary
        data = response.json()
        # print(data)

        # dataframe
        df_WTO_long = pd.DataFrame(data['Dataset'])
        # print(df_WTO_long.head())

        # simple dataframe
        df_WTO_long_simple = df_WTO_long[['ReportingEconomy', 'PartnerEconomy', 'Year', 'Value', 'ProductOrSectorCode', 'ProductOrSector']]
        print(df_WTO_long_simple.head())
        print(df_WTO_long_simple.shape)

    else:
        # Print error details if something went wrong
        print(f"Error: {response.status_code}")
        print(f"Response content: {response.text}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred with the request: {e}")

# Inspect data

In [ ]:
# Harmonized system
print(df_HS.columns)

# OEC
print(df_exports.columns)
print(df_imports.columns)

# FAO
print(df_FAO.columns)

# Wiki
print(df_WB.columns)
print(df_WTO.columns)
print(df_UNCTAD.columns)


In [ ]:
df_map = {
    'df_HS': df_HS.columns.tolist(),
    'df_exports': df_exports.columns.tolist(),
    'df_imports': df_imports.columns.tolist(),
    'df_FAO_env': df_FAO_proc_env.columns.tolist(),
    'df_FAO_prod': df_FAO_proc_prod.columns.tolist(),
    'df_FAO_forest': df_FAO_proc_forest.columns.tolist(),
    'df_WB': df_WB.columns.tolist(),
    'df_WTO': df_WTO.columns.tolist(),
    'df_UNCTAD': df_UNCTAD.columns.tolist(),
    'df_WTO_long': df_WTO_long.columns.tolist(),
    'df_countries': df_countries.columns.tolist()
}

# --- 3. Create DataFrame from the dictionary ---
# Using orient='index' makes the dictionary keys (the DataFrame names) into the rows.
# Pandas will automatically fill with NaN where lists are shorter.
column_list_df = pd.DataFrame.from_dict(df_map, orient='index')

# --- 4. Clean up the table for better readability ---
# Rename the index
column_list_df.index.name = 'DataFrame Name'

# Rename the columns from 0, 1, 2... to Column 1, Column 2, Column 3...
column_list_df.columns = [f'Column {i+1}' for i in column_list_df.columns]

print("--- DataFrame Column List Table ---")
print(column_list_df)

# Optional: Print in a Markdown format which is often easier to read
print("\n--- Markdown Format ---")
# Replace NaN with an empty string for a cleaner look in markdown
print(column_list_df.to_markdown())

# transpose
column_list_df = column_list_df.transpose()




NameError: name 'df_HS' is not defined

In [ ]:
output_dir = Path("/content/drive/MyDrive/DALAS/data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

output_path = output_dir / "dataframe_columns.csv"
column_list_df.to_csv(output_path)

print(f"DataFrame saved to {output_path}")